<a href="https://colab.research.google.com/github/Dowell-Lab/psea/blob/main/notebook_examples/read_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages we will use

In [1]:
import pandas as pd
import plotly.express as px

# read in the files we will look at

In [3]:
url="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/psea_scores_20240923-152820.adjpval.csv"
df = pd.read_csv(url, index_col=0)
url2="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/genes.csv"
genedf = pd.read_csv(url2, index_col=0)

In [ ]:
genedf["value"]=genedf["gene_id"]

# Look at the files

In [ ]:
df[["binary_attribute", "value", "NES", "p_value_BenjaminiHochberg"]]

In [ ]:
df[df["value"]=="ENSG00000159200"]

In [ ]:
df_nona = df.dropna()

# Most genes don't vary with most comorbiditys


In [ ]:
fig = px.violin(df_nona, y="p_value_BenjaminiHochberg")
fig.show()


In [ ]:
fig = px.violin(df_nona, y="p_value_BenjaminiYekutieli")
fig.show()

In [ ]:
fig = px.violin(df_nona, y="p_value_bonf")
fig.show()


In [ ]:
fig = px.violin(df_nona, y="p_value_holm")
fig.show()


# Find pairs predicted by PSEA

In [ ]:
df_nona_threshold = df_nona[df_nona["p_value_BenjaminiHochberg"]<0.1]

In [ ]:
df_nona_threshold

In [ ]:
df_nona_threshold_names = df_nona_threshold.merge(genedf, how ="inner", on="value")
df_nona_threshold_names

# How many binary_attribute do we pair

In [ ]:
df_nona_threshold_names["binary_attribute"].value_counts()

# How many values (genes) do we pair

In [ ]:
df_nona_threshold_names["gene_name"].value_counts()

# Pick a value to look at

In [ ]:
#agene = "RCAN1"
#agene = "PRMT2"
#agene = "IFNAR2"
#agene="ETS2"
agene="RUNX1"
agene_df = df_nona_threshold_names[df_nona_threshold_names["gene_name"]==agene]
agene_df = agene_df.sort_values(["NES"])
agene_df

# Pick a binary_attribute to look at

In [ ]:
acondition = "constipation_disorder"
#acondition = "strabismus"
aconditiondf = df_nona_threshold_names[df_nona_threshold_names["binary_attribute"]==acondition]
aconditiondf = aconditiondf.sort_values(["NES"])
aconditiondf

In [ ]:
high_get_aconditiondf = aconditiondf[aconditiondf["NES"]<0]
high_get_aconditiondf

In [ ]:
high_get_aconditiondf["gene_name"].to_list()

In [ ]:
high_donot_have_aconditiondf = aconditiondf[aconditiondf["NES"]>0]
high_donot_have_aconditiondf

In [ ]:
high_donot_have_aconditiondf["gene_name"].to_list()

# Are most effects of genes being to high?

In [ ]:
df_nona_threshold_names["high_get_disorder"] = df_nona_threshold_names["NES"]<0

In [ ]:
#df.groupby('Company Name')['Amount'].agg(['sum','count'])
n_genes_associated = df_nona_threshold_names.groupby('binary_attribute')["high_get_disorder"].agg(['count', 'sum'])
n_genes_associated.columns = ["total_genes_associated", "high_genes_associated"]
n_genes_associated = n_genes_associated.sort_values(["high_genes_associated"], ascending=False)
n_genes_associated["low_genes_associated"] = n_genes_associated["total_genes_associated"]-n_genes_associated["high_genes_associated"]
n_genes_associated
